In [ ]:
def add_directions(path: "json", direction = True):
    """
    This code will add directions UP_LON or DOWN_LON to each of the signals in "nodes" in the JSON file automatically.
    
    path : It is the JSON file which will be used, make sure to have a r in front of the path. 
    
    direction :  It is a boolean and depending on what the user decides, it will detemrine the directions of the odd and even
    signals. From opentraintimes, it can be seen that the even signals are going to the left whilst the odd are going to the
    right. For GWR, it is going from Paddington to Swindon, Paddington is in the rightest part, so the line goes from right to
    left, so the odd signals would be going away from London with is DOWN_LON, and the opposite for the even signals. 
    By default this case of the line going from right to left is True, however, for example with LNER which goes from left to 
    right, direction should be False. Therefore, the user has to decide this.
    
    
    DISCLAIMER: Sometimes, around the middle of the line this pattern of left signals are odd number and right are even numbers, may change.
    The user has to give a quick look accross the whole line in opentraintimes to make sure this doesn't occur, otherwise the
    directions of many signals will end up being wrong. I will think of way of how to solve this and update the code.
    
    The code will end up rewriting the JSON file and adding directions. This direction will be "" for the nodes and either 
    UP_LON or DOWN_LON for the signals.
    
    """
    import pandas as pd
    import json
    from pandas.io.json import json_normalize
    with open(path) as data_file:    
        data = json.load(data_file) 
    #These previous lines make sure to open the JSON file as a dataframe  and save it as data.
    df_nodes = pd.DataFrame(data['nodes'])
    #This transforms "nodes" in the JSON file as a dataframe.
    direction_nodes = []
    characters = "0123456789&"
    #I use characters in order to add the important part of the signals which are the numbers, e.g. "SIG:SN211",
    #the only important part is 211. Moreover "&" is also included as some signals "OL SIG:T1766 & T1779" are like this.
    #By default the code will end up only taking 1779 as the signal in this case.
    for i in df_nodes['name']:
        if i == "":
            direction_nodes.append("")
        else:
            signal = ""
            for b in i:
                if b in characters:
                    signal += b
            if len(signal) > 4:
                signal = signal.split("&")[1]
            if len(signal) > 0:
                if direction == True:
                    if int(signal)%2 == 0:
                        direction_nodes.append("UP_LON")
                    else:
                        direction_nodes.append("DOWN_LON")
                else:
                    if int(signal)%2 == 0:
                        direction_nodes.append("DOWN_LON")
                    else:
                        direction_nodes.append("UP_LON")
            else:
                direction_nodes.append("")
                
    df_nodes.insert(4,"direction",direction_nodes)
    df_nodes_json = df_nodes.to_json(orient = 'records')
    json_object = json.loads(df_nodes_json)
    data['nodes'] = json_object
    #The "nodes" in the JSON file are now this new df_nodes which has this new column for directions.
    #The next two lines rewrites the original file adding the directions.
    with open(path, 'w') as json_file:
        json.dump(data, json_file)
    

In [12]:
add_directions(r'C:\Users\Gyalpo\Downloads\nv-tracks-GWR_V4_Whole_file.json')